In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from collections import defaultdict
import re
import os

In [ ]:
script_links = []
with urlopen('https://www.livesinabox.com/friends/scripts.shtml') as res:
    soup = BeautifulSoup(res, 'html.parser')
    for anchor in soup.find_all('a'):
        if 'Episode ' in anchor.get_text() and any(s not in anchor.get_text() for s in ['Guide']):
            # print(anchor.get('href'))
            script_links.append(anchor.get('href'))

In [ ]:
BASE = 'https://www.livesinabox.com/friends/'

def get_season(link) -> int:
    if 'season' in link:
        season_num = int(link.lstrip("season").split("/")[0])
    else:
        season_num = 10
    return season_num


def get_episode(link) -> int:
    if link == 'season9/0923-0924.html':
        episode = 23
    if 'season' in link:
        episode = int(re.search('/\d*', link).group()[2:])
    else:
        episode = int(link[2:4])
    return episode

season_labeled_links = defaultdict(list)
for link in script_links:
    season_labeled_links[get_season(link)].append((get_episode(link), link))
print(season_labeled_links.values())

In [ ]:
DEST_DIR = './data/'

for season_num in season_labeled_links.keys():
    try:
        os.mkdir(DEST_DIR + "season" + str(season_num))
    except Exception as e:
        print(e)

In [ ]:
for season_num in season_labeled_links.keys():
    for _, (episode_num, script_link) in enumerate(season_labeled_links[season_num]):
        with urlopen(BASE + script_link) as res:
            soup = BeautifulSoup(res, 'html.parser')
            lines = soup.find('body').get_text().split('\n')
            lines = [line for line in lines if not any(hay in line for hay in [
                'Written by:',
                'Transcribed by:',
                'Additional transcribing by:',
                '(Note: The previously unseen parts of this episode are shown in blue text.)'
            ])]
            text = '\n'.join(lines)
            print(f"S{season_num} E{episode_num}: EXTRACTED")
            with open(DEST_DIR + f"season{str(season_num)}/episode{str(episode_num)}.txt", 'w') as f:
                f.write(text)